<a href="https://colab.research.google.com/github/nourfayed/inception_transfer_learning/blob/master/inception.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
cd gdrive/My Drive/Neural Networks/Files

/content/gdrive/My Drive/Neural Networks/Files


In [0]:
!ls

checkpoint			     the_cnn_model4.tfl.meta
inception_resnet_v2_2016_08_30.ckpt  the_cnn_model5.tfl.index
inception_resnet_v2.py		     the_cnn_model5.tfl.meta
labels.npy			     the_cnn_model6.tfl.index
labelsTrain.npy			     the_cnn_model6.tfl.meta
log				     the_cnn_model7.tfl.index
__pycache__			     the_cnn_model7.tfl.meta
swarTrain.npy			     the_cnn_model8.tfl.index
testcsv3.csv			     the_cnn_model8.tfl.meta
TestingImages.npy		     tmp
the_cnn_model1.tfl.index	     Training_images.npy
the_cnn_model1.tfl.meta		     Training_labels.npy
the_cnn_model2.tfl.index	     train.npy
the_cnn_model2.tfl.meta		     validation_images.npy
the_cnn_model4.tfl.index	     validation_labels.npy


In [0]:
import numpy as np

X_Train=[]
Y_TrainLabels=[]
X_Validation = [] #swar
Y_ValidationLabels = [] #labels
Testing_images=[]


X_Validation=np.load('validation_images.npy',allow_pickle=True)
Y_ValidationLabels=np.load('validation_labels.npy',allow_pickle=True)
Y_TrainLabels =np.load('Training_labels.npy',allow_pickle=True)
X_Train = np.load('Training_images.npy',allow_pickle=True)
Testing_images = np.load('TestingImages.npy',allow_pickle=True)

print("khalas")



khalas


In [0]:
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
import inception_resnet_v2 as incep_v2
import os
import numpy as np
from tqdm import tqdm
import matplotlib.patches as mpatches
from random import shuffle
from google.colab import files
from PIL import Image
import glob
import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression


Features_vector = []
n_rows = 299
n_cols = 299




def define_model(model, is_training):
    model.Image = tf.placeholder(tf.float32, shape=[None, n_rows, n_cols, 3])
    with incep_v2.slim.arg_scope(incep_v2.inception_resnet_v2_arg_scope()):
        model.logits, model.end_points = incep_v2.inception_resnet_v2(model.Image, is_training=False)

sess = tf.Session()

class Model_Class:
    def __init__(self, is_training):
        define_model(self, is_training=is_training)


print("2")
# ------
#----------------------------------------------------------------------------
# Create Model
# ------------------------------------------------------------------------------
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
with tf.device('/gpu:0'):
    model = Model_Class(False)

print("done")
# ------------------------------------------------------------------------------
# Load the weights and Run the classifier
# ------------------------------------------------------------------------------
#model.saver
model.saver = tf.train.Saver()
model.saver.restore(sess,  'inception_resnet_v2_2016_08_30.ckpt')
predictions = model.end_points["PreLogitsFlatten"]

print("4")

Labels=[]
for i in range (len(X_Train)):
  for j in range (len(X_Train[i])):
    img=X_Train[i][j]
    label=Y_TrainLabels[i]
    tmp = sess.run(predictions, feed_dict={model.Image: [img]})
    Features_vector.append(tmp)
    Labels.append(label)



print("5")
print(Features_vector)
print(np.shape(Features_vector))


fully_layer = fully_connected(Features_vector, 1024, activation='relu')
#fully_layer = dropout(fully_layer, 0.4)
cnn_layers = fully_connected(fully_layer, 2, activation='softmax')
cnn_layers = regression(cnn_layers, optimizer='adam', learning_rate=LR, loss='categorical_crossentropy', name='targets')
cnn_model = tflearn.DNN(cnn_layers, tensorboard_dir='log', tensorboard_verbose=3)
if (os.path.exists('cnn_model.tfl.meta')):
   cnn_model.load('./cnn_model.tfl')
else:
    cnn_model.fit({'input': Features_vector}, {'targets': Labels}, n_epoch=10,validation_set=({'input': X_Validation}, {'targets': Y_ValidationLabels}),snapshot_step=500, show_metric=True, run_id=MODEL_NAME)
    p1 = '/content/drive/My Drive/Neural Networks/Files/'
    swarFile2 = p1 +'cnn_model.tfl'    
    cnn_model.save(swarFile2)




For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

2
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.flatten instead.
done
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from inception_resnet_v2_2016_08_30.ckpt
4
5
[array([[ 0.       ,  0.9909607,  0.5673769, ..., 19.823788 , 53.22419  ,
        50.267864 ]], dtype=float32), array([[ 0.        ,  0.35623604,  0.9044041 , ..., 33.54203   ,
        76.20007   , 36.032265  ]], dtype=float32), array([[ 0.73806995,  1.2941161 ,  0.370345  , ..., 23.729874  ,
        66.81203   , 53.32562   ]], dtype=float32), array([[6.1482084e-03, 8.5694551e-01, 4.0335903e-01, ..., 3.8942242e+01,
        9.7536949e+01, 6.1798626e+01

In [0]:
from PIL import Image
import glob
import cv2

path = '/content/drive/My Drive/Neural Networks/train/*'
pathes = glob.glob(path)
X=[]
Y=[]
 label_name = ''
if (os.path.exists('swar.npy')): # If you have already created the dataset:
    X =np.load('swar.npy')
    Y = np.load('labels.npy')
else:
  for i in range (len(pathes)):
    s = pathes[i]+"/*.jpg"
    tmp=glob.glob(s)
    x=[]
    for j in range (len(tmp)):
      print(len(tmp))
      print(j)
      img = cv2.imread(tmp[j])
      height, width,faks = img.shape
      if height>0 and width>0: 
        img = cv2.resize(img, (299, 299))
        print(tmp[j])
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # Make it RGB
        x.append(img)
        X.append(x)
        label_name=s[46:(len(s)-6)]
      Y.append(label_name)
  np.save('swar.npy', X)
  np.save('labels.npy',Y)    
print(Y)
print(X)
print(np.shape(Y))


In [0]:
import tensorflow as tf  

#da betengan msh shaghal bgad
def newnetwork(Feature_vector,label,IsTrainng ): 
    conv1 = tf.layers.conv2d(inputs=Feature_vector,filters=32,kernel_size=[5, 5],padding="same",activation=tf.nn.relu)
    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

    conv2 = tf.layers.conv2d(inputs=pool1,filters=64,kernel_size=[5, 5],padding="same",activation=tf.nn.relu)
    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

    conv3 = tf.layers.conv2d(inputs=pool2,filters=128,kernel_size=[5, 5],padding="same",activation=tf.nn.relu)
    pool3 = tf.layers.max_pooling2d(inputs=conv3, pool_size=[2, 2], strides=2)

    conv4 = tf.layers.conv2d(inputs=pool3,filters=64,kernel_size=[5, 5],padding="same",activation=tf.nn.relu)
    pool4 = tf.layers.max_pooling2d(inputs=conv4, pool_size=[2, 2], strides=2)

    conv5 = tf.layers.conv2d(inputs=pool4,filters=32,kernel_size=[5, 5],padding="same",activation=tf.nn.relu)
    pool5 = tf.layers.max_pooling2d(inputs=conv5, pool_size=[2, 2], strides=2)

    conv6 = tf.layers.conv2d(inputs=pool5,filters=64,kernel_size=[5, 5],padding="same",activation=tf.nn.relu)
    pool6 = tf.layers.max_pooling2d(inputs=conv6, pool_size=[2, 2], strides=2)

    x,y,faks=pool6.shape
    pool_flat = tf.reshape(pool6, [-1, x * y * 64])  #da -1 ya3ni hay3raf dynamically eh l batch size
    dense = tf.layers.dense(inputs=pool_flat, units=1024, activation=tf.nn.relu) #1024 da olayel?
    trainingMode=true
    dropout = tf.layers.dropout(inputs=dense, rate=0.4, training=trainingMode) #40% will be randomlly dropped out
    logits = tf.layers.dense(inputs=dropout, units=10)

    logits = tf.layers.dense(inputs=dropout, units=10)

    predictions = { "classes": tf.argmax(input=logits, axis=1),"probabilities": tf.nn.softmax(logits, name="softmax_tensor")}
return predictions

"""
if trainingMode==false :  #ya3ni han-predict
  predictedClass=

loss=tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)  #3aiza l labels hna
"""



#shwyat hagta 3yzen n7sbhaa

cost = tf.reduce_mean(tf.losses.mean_squared_error(labels=y, logits=yhat))
optimizer = tf.train.GradientDescentOptimizer(learning_rate)
train_op = optimizer.minimize(cost)


# Initializing the variables
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(training_epochs):
        # Stochasting Gradient Descent
        for i in range(len(X_train)):
            summary = sess.run(train_op, feed_dict={X: X_train[i], y: y_train[i]})
        current_output = sess.run(ypredict, feed_dict={X: X_test})
        #test_accuracy = np.mean(np.argmax(y_test, axis=1) == current_output)
    sess.close()


In [0]:
from google.colab import files
src = list(files.upload().values())[0]
open('inception_resnet_v2.py','wb').write(src)
import inception_resnet_v2

In [0]:
from PIL import Image
import glob
import cv2
import numpy as np
import os

path = '/content/drive/My Drive/Neural Networks/test/*.jpg'
pathes = glob.glob(path)
testing_images=[]


for i in range (len(pathes)):
  img = cv2.imread(tmp[j])
  height, width,faks = img.shape
  if height>0 and width>0: 
    img = cv2.resize(img, (299, 299))
    print(tmp[j])
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # Make it RGB
    testing_images.append(img)
p1 = '/content/drive/My Drive/Neural Networks/Files/'
swarFile2 = p1 +'testing_images.npy'    
np.save(swarFile2,testing_images)
   
print(np.shape(testing_images))


(0,)
